In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import load_counties_geojson

counties_geojson = load_counties_geojson()

In [ ]:
from sportwarz import load_counties_data

co_data_frame = load_counties_data(counties_geojson)

### Blank Sandbox

In [ ]:
from ipyleaflet import Map, GeoJSON, Popup, FullScreenControl, basemaps

from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)

map = render_map({ }, None, counties_geojson, co_data_frame)

map

In [ ]:
from sportwarz import Team, Coordinates, LeagueJson,\
    calculate_distances, create_show_teams, compute_shares, add_team,\
    compute_output_dataframes, reset_county_styles, heatmap_counties

import json
import codecs
import ipywidgets as widgets

leagues = { }

league_dropdown_options = []

league_input = widgets.Dropdown(options = league_dropdown_options)
name_input = widgets.Text()
venue_input = widgets.Text()
L_input = widgets.FloatText()
S_input = widgets.FloatText()
N_input = widgets.FloatText()
lat_input = widgets.FloatText()
lon_input = widgets.FloatText()
color_input = widgets.ColorPicker()
state_input = widgets.Text()
add_button = widgets.Button(description="Add Team")
error_message = widgets.Label(style = { "color" : "red" })

out = widgets.Output()
out.clear_output()

def add_new_team(button):
    button.disabled=True
    try:
        if not league_input.value in leagues:
            error_message.value = "League not specified"
            return
        if not name_input.value.strip():
            error_message.value = "Name not specified"
            return
        if not L_input.value:
            error_message.value = "L not specified"
            return
        if not S_input.value:
            error_message.value = "S not specified"
            return
        if not N_input.value:
            error_message.value = "N not specified"
            return
        if not venue_input.value:
            error_message.value = "Venue not specified"
            return
        if not lon_input.value:
            error_message.value = "Lon not specified"
            return
        if not lat_input.value:
           error_message.value = "Lat not specified"
           return
    
        newTeam = Team(
            name = name_input.value.strip(),
            L = L_input.value,
            S = S_input.value,
            N = N_input.value,
            venue = venue_input.value,
            state = state_input.value,
            color = color_input.value,
            coordinates = Coordinates(
                lat = lat_input.value,
                lon = lon_input.value
        ))
        add_team(co_data_frame, leagues, league_input.value, [newTeam])

        reset_county_styles(counties_geojson)
        heatmap_counties(leagues, counties_geojson, league_input.value)
        map.remove_layer(map.layers[1])
        layer = GeoJSON(data = counties_geojson, 
            hover_style = {"fillColor": "white"}
        )    
        layer.on_click(create_show_teams(map, leagues, co_data_frame))
        map.add(layer)
    finally:
        button.disabled=False

add_button.on_click(add_new_team)

add_panel = widgets.VBox([widgets.GridBox([
    widgets.Label(value='League:'), league_input,
    widgets.Label(value='Name:'), name_input,
    widgets.Label(value='Venue:'), venue_input,
    widgets.Label(value='L:'), L_input,
    widgets.Label(value='S:'), S_input,
    widgets.Label(value='N:'), N_input,
    widgets.Label(value='Lat:'), lat_input,
    widgets.Label(value='Lon:'), lon_input,
    widgets.Label(value='Color:'), color_input,
    widgets.Label(value='State:'), state_input,
    add_button
],layout=widgets.Layout(grid_template_columns="repeat(2, 100px)")),
    error_message
])

upload = widgets.FileUpload(
    accept='.json', 
    multiple= False  
)

@out.capture()
def on_upload_change(change):
    upload.disabled = True
    try:
        file_info = change["new"][0]
        content = codecs.decode(file_info.content, encoding="utf-8")
        league_json: LeagueJson = json.loads(content)
        league_name = league_json["league_name"]
        if not league_name in leagues:
            leagues[league_name] = {}
        leagues[league_name]["json"] = league_json
        calculate_distances(leagues, co_data_frame) 
        compute_shares(leagues, co_data_frame)  
        compute_output_dataframes(leagues, co_data_frame) 
    
        reset_county_styles(counties_geojson)
        heatmap_counties(leagues, counties_geojson, league_name)
        map.remove_layer(map.layers[1])
        layer = GeoJSON(data = counties_geojson, 
            hover_style = {"fillColor": "white"}
        )    
        layer.on_click(create_show_teams(map, leagues, co_data_frame))
        map.add(layer)
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)    
        league_input.options = league_dropdown_options
    finally:
        upload.disabled=False   

upload.observe(on_upload_change, names=["value"])

upload_panel = widgets.GridBox([
    widgets.Label(value='File Upload:'), upload,    
], layout=widgets.Layout(grid_template_columns="repeat(2, 100px)"))

tab = widgets.Tab()
tab.children = [upload_panel, add_panel]
tab.titles =   ["Upload", "Add"]
tab


In [ ]:
leagues["NHL"]["json"]["teams"]